In [ ]:
import os

directory = '/kaggle/input/learntools/'
for filename in os.listdir(directory):
    print(os.path.join(directory, filename))

# Tiền xử lý dữ liệu cho model bounding box

In [ ]:
# Import các thư viện cần thiết
import os
import pandas as pd
import cv2
import numpy as np

# Đọc dữ liệu từ các tệp CSV
test_df = pd.read_csv("/kaggle/input/learntools/test/_annotations.csv")
train_df = pd.read_csv("/kaggle/input/learntools/train/_annotations.csv")
valid_df = pd.read_csv("/kaggle/input/learntools/valid/_annotations.csv")

# Hàm đọc ảnh và xử lý bounding box
def process_data(dataframe, data_dir):
    images = []
    labels = []
    for index, row in dataframe.iterrows():
        filename = row['filename']
        image_path = os.path.join(data_dir, filename)
        image = cv2.imread(image_path)  # Đọc ảnh
        if image is not None:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Chuyển đổi màu sắc từ BGR sang RGB
            images.append(image)
            
            # Chuẩn hóa bounding box
            xmin = row['xmin']
            ymin = row['ymin']
            xmax = row['xmax']
            ymax = row['ymax']
            width = row['width']  # Chiều rộng ảnh
            height = row['height']  # Chiều cao ảnh
            # Chuẩn hóa tọa độ bounding box
            xmin_norm = xmin / width
            ymin_norm = ymin / height
            xmax_norm = xmax / width
            ymax_norm = ymax / height
            
            labels.append([xmin_norm, ymin_norm, xmax_norm, ymax_norm])
    
    return np.array(images), np.array(labels)

# Xử lý dữ liệu cho tập train, validation và test
train_images, train_labels = process_data(train_df, "/kaggle/input/learntools/train/")
valid_images, valid_labels = process_data(valid_df, "/kaggle/input/learntools/valid/")
test_images, test_labels = process_data(test_df, "/kaggle/input/learntools/test/")

# **Tự xây dựng mô hình**

## **1.LeNet5**

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, AveragePooling2D, Flatten, Dense

def LeNet5():
    model = Sequential()

    # C1 Convolutional layer
    model.add(Conv2D(6, kernel_size=(5, 5), activation='tanh', input_shape=(224, 224, 3), padding='same', name='C1'))

    # S2 Sub-sampling layer (pooling layer)
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='S2'))

    # C3 Convolutional layer
    model.add(Conv2D(16, kernel_size=(5, 5), activation='tanh', padding='valid', name='C3'))

    # S4 Sub-sampling layer (pooling layer)
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='S4'))

    # C5 Convolutional layer
    model.add(Conv2D(120, kernel_size=(5, 5), activation='tanh', padding='valid', name='C5'))

    model.add(Flatten(name='Flatten'))

    # FC6 Fully connected
    model.add(Dense(84, activation='tanh', name='FC6'))

    # FC7 output
    model.add(Dense(4, activation='linear', name='FC7'))  # Số lượng lớp đầu ra được thay đổi thành 4 và sử dụng activation là 'linear'

    return model

if __name__ == '__main__':
    model_lenet5 = LeNet5()
    print(model_lenet5.summary())


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

# Xây dựng mô hình LeNet5
model_lenet5 = LeNet5()
model_lenet5.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Huấn luyện mô hình LeNet5
checkpoint_lenet5 = ModelCheckpoint("lenet5_checkpoint.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history_lenet5 = model_lenet5.fit(train_images, train_labels, batch_size=64, epochs=10, validation_data=(valid_images, valid_labels), callbacks=[checkpoint_lenet5])

# Đánh giá mô hình LeNet5 trên tập kiểm tra
test_loss_lenet5, test_accuracy_lenet5 = model_lenet5.evaluate(test_images, test_labels)
print("LeNet5 - Test Loss:", test_loss_lenet5)
print("LeNet5 - Test Accuracy:", test_accuracy_lenet5)

# Trực quan hóa kết quả huấn luyện của mô hình LeNet5
plt.plot(history_lenet5.history['accuracy'], label='accuracy')
plt.plot(history_lenet5.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('LeNet5 - Training and Validation Accuracy')
plt.show()

plt.plot(history_lenet5.history['loss'], label='loss')
plt.plot(history_lenet5.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('LeNet5 - Training and Validation Loss')
plt.show()

# Lưu mô hình đã huấn luyện
model_lenet5.save("lenet5_model.keras")


**Thay đổi kiến trúc mô hình LeNet5**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense, Dropout

def LeNet5():
    model = Sequential()

    # C1 Convolutional layer
    model.add(Conv2D(6, kernel_size=(5, 5), activation='tanh', input_shape=(224, 224, 3), padding='same', name='C1'))

    # S2 Sub-sampling layer (pooling layer)
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='S2'))

    # C3 Convolutional layer
    model.add(Conv2D(16, kernel_size=(5, 5), activation='tanh', padding='valid', name='C3'))

    # S4 Sub-sampling layer (pooling layer)
    model.add(AveragePooling2D(pool_size=(2, 2), strides=(2, 2), name='S4'))

    # C5 Convolutional layer
    model.add(Conv2D(120, kernel_size=(5, 5), activation='tanh', padding='valid', name='C5'))

    model.add(Flatten(name='Flatten'))

    # FC6 Fully connected
    model.add(Dense(84, activation='tanh', name='FC6'))

    # Thêm lớp Dropout để giảm overfitting
    model.add(Dropout(0.5))

    # FC7 output
    model.add(Dense(4, activation='linear', name='FC7'))

    return model

In [ ]:
 model_lenet5 = LeNet5()
print(model_lenet5.summary())

from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

# Xây dựng mô hình LeNet5
model_lenet5 = LeNet5()
model_lenet5.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Huấn luyện mô hình LeNet5
checkpoint_lenet5 = ModelCheckpoint("lenet5_checkpoint.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history_lenet5 = model_lenet5.fit(train_images, train_labels, batch_size=64, epochs=100, validation_data=(valid_images, valid_labels), callbacks=[checkpoint_lenet5])

# Đánh giá mô hình LeNet5 trên tập kiểm tra
test_loss_lenet5, test_accuracy_lenet5 = model_lenet5.evaluate(test_images, test_labels)
print("LeNet5 - Test Loss:", test_loss_lenet5)
print("LeNet5 - Test Accuracy:", test_accuracy_lenet5)

# Trực quan hóa kết quả huấn luyện của mô hình LeNet5
plt.plot(history_lenet5.history['accuracy'], label='accuracy')
plt.plot(history_lenet5.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('LeNet5 - Training and Validation Accuracy')
plt.show()

plt.plot(history_lenet5.history['loss'], label='loss')
plt.plot(history_lenet5.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('LeNet5 - Training and Validation Loss')
plt.show()

# Lưu mô hình đã huấn luyện
model_lenet5.save("lenet5_model_v2.keras")

## **2. AlexNet**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout

def AlexNet():
    model = Sequential()

    # Tầng 1 Convolutional Layer
    model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3), padding='valid', name='Conv1'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='MaxPool1'))
    model.add(BatchNormalization(name='BN1'))

    # Tầng 2 Convolutional Layer
    model.add(Conv2D(256, kernel_size=(5, 5), activation='relu', padding='same', name='Conv2'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='MaxPool2'))
    model.add(BatchNormalization(name='BN2'))

    # Tầng 3 Convolutional Layer
    model.add(Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same', name='Conv3'))
    model.add(BatchNormalization(name='BN3'))

    # Tầng 4 Convolutional Layer
    model.add(Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same', name='Conv4'))
    model.add(BatchNormalization(name='BN4'))

    # Tầng 5 Convolutional Layer
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', name='Conv5'))
    model.add(BatchNormalization(name='BN5'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='MaxPool3'))

    model.add(Flatten(name='Flatten'))

    # Tầng 6 FC
    model.add(Dense(4096, activation='relu', name='FC1'))
    model.add(Dropout(0.5, name='Dropout1'))

    # Tầng 7 FC
    model.add(Dense(4096, activation='relu', name='FC2'))
    model.add(Dropout(0.5, name='Dropout2'))

    # Tầng 8 FC
    model.add(Dense(4, activation='softmax', name='FC3'))  # Số lượng lớp đầu ra được thay đổi thành 4

    return model

if __name__ == '__main__':
    model_alexnet = AlexNet()
    print(model_alexnet.summary())


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

# Xây dựng mô hình AlexNet
model_alexnet = AlexNet()
model_alexnet.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Huấn luyện mô hình AlexNet
checkpoint_alexnet = ModelCheckpoint("alexnet_checkpoint.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history_alexnet = model_alexnet.fit(train_images, train_labels, batch_size=64, epochs=10, validation_data=(valid_images, valid_labels), callbacks=[checkpoint_alexnet])

# Đánh giá mô hình AlexNet trên tập kiểm tra
test_loss_alexnet, test_accuracy_alexnet = model_alexnet.evaluate(test_images, test_labels)
print("AlexNet - Test Loss:", test_loss_alexnet)
print("AlexNet - Test Accuracy:", test_accuracy_alexnet)

# Trực quan hóa kết quả huấn luyện của mô hình AlexNet
plt.plot(history_alexnet.history['accuracy'], label='accuracy')
plt.plot(history_alexnet.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('AlexNet - Training and Validation Accuracy')
plt.show()

plt.plot(history_alexnet.history['loss'], label='loss')
plt.plot(history_alexnet.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('AlexNet - Training and Validation Loss')
plt.show()

# Lưu mô hình đã huấn luyện
model_alexnet.save("alexnet_model.keras")

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense, Dropout

def ImprovedAlexNet():
    model = Sequential()

    # Tầng 1 Convolutional Layer
    model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3), padding='valid', name='Conv1'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='MaxPool1'))
    model.add(BatchNormalization(name='BN1'))

    # Tầng 2 Convolutional Layer
    model.add(Conv2D(256, kernel_size=(5, 5), activation='relu', padding='same', name='Conv2'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='MaxPool2'))
    model.add(BatchNormalization(name='BN2'))

    # Tầng 3 Convolutional Layer
    model.add(Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same', name='Conv3'))
    model.add(BatchNormalization(name='BN3'))

    # Tầng 4 Convolutional Layer
    model.add(Conv2D(384, kernel_size=(3, 3), activation='relu', padding='same', name='Conv4'))
    model.add(BatchNormalization(name='BN4'))

    # Tầng 5 Convolutional Layer
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', name='Conv5'))
    model.add(BatchNormalization(name='BN5'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='MaxPool3'))

    model.add(Flatten(name='Flatten'))

    # Tầng 6 FC
    model.add(Dense(4096, activation='relu', name='FC1'))
    model.add(Dropout(0.5, name='Dropout1'))

    # Tầng 7 FC
    model.add(Dense(4096, activation='relu', name='FC2'))
    model.add(Dropout(0.5, name='Dropout2'))

    # Tầng 8 FC
    model.add(Dense(4, activation='softmax', name='FC3'))  # Số lượng lớp đầu ra được thay đổi thành 4

    return model

if __name__ == '__main__':
    model_alexnet = ImprovedAlexNet()
    print(model_alexnet.summary())

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

# Xây dựng mô hình AlexNet
model_alexnet = AlexNet()
model_alexnet.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Huấn luyện mô hình AlexNet
checkpoint_alexnet = ModelCheckpoint("alexnet_checkpoint.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history_alexnet = model_alexnet.fit(train_images, train_labels, batch_size=64, epochs=100, validation_data=(valid_images, valid_labels), callbacks=[checkpoint_alexnet])

# Đánh giá mô hình AlexNet trên tập kiểm tra
test_loss_alexnet, test_accuracy_alexnet = model_alexnet.evaluate(test_images, test_labels)
print("AlexNet - Test Loss:", test_loss_alexnet)
print("AlexNet - Test Accuracy:", test_accuracy_alexnet)

# Trực quan hóa kết quả huấn luyện của mô hình AlexNet
plt.plot(history_alexnet.history['accuracy'], label='accuracy')
plt.plot(history_alexnet.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('AlexNet - Training and Validation Accuracy')
plt.show()

plt.plot(history_alexnet.history['loss'], label='loss')
plt.plot(history_alexnet.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('AlexNet - Training and Validation Loss')
plt.show()

# Lưu mô hình đã huấn luyện
model_alexnet.save("alexnet_model_v2.keras")

## **3. VGG16**

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def VGG16():
    model = Sequential()

    # Khối VGG 1
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(224, 224, 3), name='conv2d'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_1'))
    model.add(MaxPooling2D(pool_size=2, strides=2, name='max_pooling2d'))

    # Khối VGG 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_2'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_3'))
    model.add(MaxPooling2D(pool_size=2, strides=2, name='max_pooling2d_1'))

    # Khối VGG 3
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_4'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_5'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_6'))
    model.add(MaxPooling2D(pool_size=2, strides=2, name='max_pooling2d_2'))

    # Khối VGG 4
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_7'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_8'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_9'))
    model.add(MaxPooling2D(pool_size=2, strides=2, name='max_pooling2d_3'))

    # Khối VGG 5
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_10'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_11'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_12'))
    model.add(MaxPooling2D(pool_size=2, strides=2, name='max_pooling2d_4'))

    model.add(Flatten(name='flatten'))

    # Fully connected layer
    model.add(Dense(4096, activation='relu', name='dense'))
    model.add(Dropout(0.5, name='dropout'))

    model.add(Dense(4096, activation='relu', name='dense_1'))
    model.add(Dropout(0.5, name='dropout_1'))

    model.add(Dense(4, activation='linear', name='dense_2'))  # Số lượng lớp đầu ra được thay đổi thành 4 và sử dụng activation là 'linear'

    return model

if __name__ == '__main__':
    model_vgg16 = VGG16()
    print(model_vgg16.summary())


In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

# Xây dựng mô hình VGG16
model_vgg16 = VGG16()
model_vgg16.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Huấn luyện mô hình VGG16
checkpoint_vgg16 = ModelCheckpoint("vgg16_checkpoint.keras", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history_vgg16 = model_vgg16.fit(train_images, train_labels, batch_size=64, epochs=10, validation_data=(valid_images, valid_labels), callbacks=[checkpoint_vgg16])

# Đánh giá mô hình VGG16 trên tập kiểm tra
test_loss_vgg16, test_accuracy_vgg16 = model_vgg16.evaluate(test_images, test_labels)
print("VGG16 - Test Loss:", test_loss_vgg16)
print("VGG16 - Test Accuracy:", test_accuracy_vgg16)

# Trực quan hóa kết quả huấn luyện của mô hình VGG16
plt.plot(history_vgg16.history['accuracy'], label='accuracy')
plt.plot(history_vgg16.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('VGG16 - Training and Validation Accuracy')
plt.show()

plt.plot(history_vgg16.history['loss'], label='loss')
plt.plot(history_vgg16.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('VGG16 - Training and Validation Loss')
plt.show()

# Lưu mô hình đã huấn luyện
model_vgg16.save("vgg16_model.keras")

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau

def VGG16():
    model = Sequential()

    # Khối VGG 1
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(224, 224, 3), name='conv2d'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_1'))
    model.add(MaxPooling2D(pool_size=2, strides=2, name='max_pooling2d'))

    # Khối VGG 2
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_2'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_3'))
    model.add(MaxPooling2D(pool_size=2, strides=2, name='max_pooling2d_1'))

    # Khối VGG 3
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_4'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_5'))
    model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_6'))
    model.add(MaxPooling2D(pool_size=2, strides=2, name='max_pooling2d_2'))

    # Khối VGG 4
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_7'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_8'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_9'))
    model.add(MaxPooling2D(pool_size=2, strides=2, name='max_pooling2d_3'))

    # Khối VGG 5
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_10'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_11'))
    model.add(Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same', name='conv2d_12'))
    model.add(MaxPooling2D(pool_size=2, strides=2, name='max_pooling2d_4'))

    model.add(Flatten(name='flatten'))

    # Fully connected layer
    model.add(Dense(4096, activation='relu', name='dense'))
    model.add(Dropout(0.5, name='dropout'))

    model.add(Dense(4096, activation='relu', name='dense_1'))
    model.add(Dropout(0.5, name='dropout_1'))

    model.add(Dense(4, activation='linear', name='bbox_regression'))

    return model

# Xây dựng mô hình
model = VGG16()

# Khởi tạo optimizer
initial_learning_rate = 0.001
optimizer = Adam(learning_rate=initial_learning_rate)

# Compile model
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

# Define learning rate schedule callback
lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1)

# Train model
history = model.fit(train_images, train_labels, batch_size=64, epochs=100, validation_data=(valid_images, valid_labels), callbacks=[lr_schedule])

# Evaluate model on test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


In [ ]:
import matplotlib.pyplot as plt

# Trực quan hóa kết quả huấn luyện của mô hình VGG16
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('VGG16 - Training and Validation Accuracy')
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('VGG16 - Training and Validation Loss')
plt.show()

# Lưu mô hình đã huấn luyện
model.save("vgg16_model_v2.keras")

## **4. CNN**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

def create_cnn_model():
    model = Sequential()

    # Xây dựng các tầng convolution và pooling
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same', name='Conv2D_1', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D((2, 2), name='MaxPool2D_1'))

    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', name='Conv2D_2'))
    model.add(MaxPooling2D((2, 2), name='MaxPool2D_2'))

    # Flatten để chuyển từ tensor 3D sang vector 1D
    model.add(Flatten())

    # Xây dựng các tầng fully connected
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))  # Dropout để tránh overfitting

    # Tầng output với số lượng lớp là 4 (output)
    model.add(Dense(4, activation='softmax'))

    return model

# Tạo mô hình
model_cnn = create_cnn_model()

# Hiển thị summary của mô hình để kiểm tra số lượng tham số
model_cnn.summary()

In [ ]:
# Compile model
model_cnn.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])

# Huấn luyện mô hình
history = model_cnn.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(valid_images, valid_labels))

# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = model_cnn.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
import matplotlib.pyplot as plt

# Trực quan hóa kết quả huấn luyện của mô hình CNN
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('CNN - Training and Validation Accuracy')
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('CNN - Training and Validation Loss')
plt.show()

# Lưu mô hình đã huấn luyện
model.save("cnn_model.keras")

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

def create_improved_cnn_model():
    model = Sequential()

    # Tăng số lượng lớp Convolution và Pooling
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same', name='Conv2D_1', input_shape=(224, 224, 3)))
    model.add(MaxPooling2D((2, 2), name='MaxPool2D_1'))
    
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='Conv2D_2'))
    model.add(MaxPooling2D((2, 2), name='MaxPool2D_2'))

    model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='Conv2D_3'))
    model.add(MaxPooling2D((2, 2), name='MaxPool2D_3'))

    # Flatten để chuyển từ tensor 3D sang vector 1D
    model.add(Flatten())

    # Thêm các lớp Dense với Dropout để giảm overfitting
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(0.5))
    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))

    # Lớp output với số lượng lớp là 4 (output)
    model.add(Dense(4, activation='softmax'))

    return model

# Tạo mô hình CNN cải tiến
improved_model_cnn = create_improved_cnn_model()

# Compile model
optimizer = Adam(learning_rate=0.001)
improved_model_cnn.compile(optimizer=optimizer, loss='mean_squared_error', metrics=['accuracy'])

# Sử dụng ReduceLROnPlateau callback để giảm learning rate khi đạt một ngưỡng
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-7, verbose=1)

# Huấn luyện mô hình
history = improved_model_cnn.fit(train_images, train_labels, epochs=100, batch_size=32, 
                                  validation_data=(valid_images, valid_labels), callbacks=[lr_scheduler])

# Đánh giá mô hình trên tập kiểm tra
test_loss, test_accuracy = improved_model_cnn.evaluate(test_images, test_labels)
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

In [ ]:
# Trực quan hóa kết quả huấn luyện của mô hình CNN cải tiến
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')
plt.title('Improved CNN - Training and Validation Accuracy')
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label='val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(loc='upper right')
plt.title('Improved CNN - Training and Validation Loss')
plt.show()

# Lưu mô hình đã huấn luyện
improved_model_cnn.save("cnn_model_v2.keras")